In [1]:
%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score


from collections import defaultdict
from scipy.stats.stats import pearsonr
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import pdist, squareform
from scipy.stats import mode
from scipy.spatial.distance import pdist
from scipy.cluster.hierarchy import linkage, dendrogram

In [2]:
df = pd.read_csv('training.csv')

In [3]:
df['MMRAcquisitionAuctionAveragePrice'].fillna(df['MMRAcquisitionAuctionAveragePrice'].median(), inplace = True)
df['MMRAcquisitionAuctionCleanPrice'].fillna(df['MMRAcquisitionAuctionCleanPrice'].median(), inplace = True)
df['MMRAcquisitionRetailAveragePrice'].fillna(df['MMRAcquisitionRetailAveragePrice'].median(), inplace = True)
df['MMRAcquisitonRetailCleanPrice'].fillna(df['MMRAcquisitonRetailCleanPrice'].median(), inplace = True)
df['MMRCurrentAuctionAveragePrice'].fillna(df['MMRCurrentAuctionAveragePrice'].median(), inplace = True)
df['MMRCurrentAuctionCleanPrice'].fillna(df['MMRCurrentAuctionCleanPrice'].median(), inplace = True)
df['MMRCurrentRetailAveragePrice'].fillna(df['MMRCurrentRetailAveragePrice'].median(), inplace = True)
df['MMRCurrentRetailCleanPrice'].fillna(df['MMRCurrentRetailCleanPrice'].median(), inplace = True)

In [4]:
df['MMR_factor'] = ( df['MMRAcquisitionAuctionAveragePrice'] + df['MMRAcquisitonRetailCleanPrice'] + df['MMRCurrentAuctionCleanPrice'] + df['MMRCurrentRetailAveragePrice']) / 4

In [5]:
df.columns

Index(['RefId', 'IsBadBuy', 'PurchDate', 'Auction', 'VehYear', 'VehicleAge',
       'Make', 'Model', 'Trim', 'SubModel', 'Color', 'Transmission',
       'WheelTypeID', 'WheelType', 'VehOdo', 'Nationality', 'Size',
       'TopThreeAmericanName', 'MMRAcquisitionAuctionAveragePrice',
       'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice',
       'MMRAcquisitonRetailCleanPrice', 'MMRCurrentAuctionAveragePrice',
       'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailAveragePrice',
       'MMRCurrentRetailCleanPrice', 'PRIMEUNIT', 'AUCGUART', 'BYRNO',
       'VNZIP1', 'VNST', 'VehBCost', 'IsOnlineSale', 'WarrantyCost',
       'MMR_factor'],
      dtype='object')

In [6]:
#BINNING NUMERICAL VALUES
df['VehBCost_bin'] = pd.cut(df.VehBCost.astype(int), 10, right = False)
df['VehOdo_bin'] = pd.cut(df.VehOdo.astype(int), 10, right = False)
df['MMR_factor_bin'] = pd.cut(df.MMR_factor.astype(int), 10, right = False)
df['WarrantyCost_bin'] = pd.cut(df.WarrantyCost.astype(int), 10, right = False)

In [7]:
df.columns

Index(['RefId', 'IsBadBuy', 'PurchDate', 'Auction', 'VehYear', 'VehicleAge',
       'Make', 'Model', 'Trim', 'SubModel', 'Color', 'Transmission',
       'WheelTypeID', 'WheelType', 'VehOdo', 'Nationality', 'Size',
       'TopThreeAmericanName', 'MMRAcquisitionAuctionAveragePrice',
       'MMRAcquisitionAuctionCleanPrice', 'MMRAcquisitionRetailAveragePrice',
       'MMRAcquisitonRetailCleanPrice', 'MMRCurrentAuctionAveragePrice',
       'MMRCurrentAuctionCleanPrice', 'MMRCurrentRetailAveragePrice',
       'MMRCurrentRetailCleanPrice', 'PRIMEUNIT', 'AUCGUART', 'BYRNO',
       'VNZIP1', 'VNST', 'VehBCost', 'IsOnlineSale', 'WarrantyCost',
       'MMR_factor', 'VehBCost_bin', 'VehOdo_bin', 'MMR_factor_bin',
       'WarrantyCost_bin'],
      dtype='object')

In [8]:
#SELECT THE COLUMNS I AM INTERESTED IN
df_new = df[['Auction','VehicleAge','Make','Model','Trim','Color','Transmission','WheelType','Nationality','Size',
            'TopThreeAmericanName','VNZIP1','VNST','VehBCost_bin', 'VehOdo_bin', 'MMR_factor_bin', 'WarrantyCost_bin']]

In [9]:
df_new.columns

Index(['Auction', 'VehicleAge', 'Make', 'Model', 'Trim', 'Color',
       'Transmission', 'WheelType', 'Nationality', 'Size',
       'TopThreeAmericanName', 'VNZIP1', 'VNST', 'VehBCost_bin', 'VehOdo_bin',
       'MMR_factor_bin', 'WarrantyCost_bin'],
      dtype='object')

In [10]:
#FIX MISSING VALUES
df_new.isnull().sum()

Auction                    0
VehicleAge                 0
Make                       0
Model                      0
Trim                    1911
Color                      7
Transmission               8
WheelType               2577
Nationality                4
Size                       4
TopThreeAmericanName       4
VNZIP1                     0
VNST                       0
VehBCost_bin               0
VehOdo_bin                 0
MMR_factor_bin             0
WarrantyCost_bin           0
dtype: int64

In [11]:
df_new.dropna(subset = ['Transmission','Color','Nationality','Size','TopThreeAmericanName'], inplace=True)

C:\Users\bonin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df_new.Trim.isnull().sum()

1910

In [13]:
df_new.Trim.value_counts()

Bas    11210
LS      8190
SE      7471
SXT     3058
LT      2849
LX      1919
Tou     1828
EX      1703
SEL     1093
XLT     1085
GLS     1083
Spo      997
SLT      946
Lim      761
Lar      697
ST       696
GT       643
XLS      623
2        586
ES       553
S        493
SES      481
ZX4      471
LE       460
SLE      416
i        385
Cla      375
XL       361
Nor      348
GL       327
       ...  
GX         6
L          6
LW2        6
C          5
Hig        5
SE-        5
Ral        4
Spe        4
XRS        4
eC         4
ZTW        4
Ove        4
ZR2        3
RS         3
Car        3
Ent        3
250        3
Hyb        2
XS         2
GXP        2
T5         2
Dur        2
JLX        1
LL         1
Si         1
JLS        1
Maz        1
Ult        1
Xsp        1
Out        1
Name: Trim, Length: 131, dtype: int64

In [14]:
df_new.Trim.fillna('Bas', inplace = True, limit=801)

C:\Users\bonin\Anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [15]:
df_new.Trim.fillna('LS', inplace = True, limit=580)

In [16]:
df_new.Trim.fillna('SE', inplace = True, limit=530)

In [17]:
df_new.Trim.isnull().sum()

0

In [18]:
df_new.WheelType.value_counts()

Alloy      28733
Covers     26482
Special      589
Name: WheelType, dtype: int64

In [19]:
df_new.WheelType.isnull().sum()

2570

In [20]:
df_new.WheelType.fillna('Alloy', inplace = True, limit=1377)

In [21]:
df_new.WheelType.isnull().sum()

1193

In [22]:
df_new.WheelType.fillna('Covers', inplace = True, limit=1200)

In [23]:
df_new.WheelType.isnull().sum()

0

In [24]:
df_new.Color.isnull().sum()
df_new.Transmission.isnull().sum()
df_new.Nationality.isnull().sum()
df_new.Size.isnull().sum()
df_new.TopThreeAmericanName.isnull().sum()
df_new.Trim.isnull().sum()
df_new.WheelType.isnull().sum()

0

In [25]:
#WHY COLOR,TRANSMISSION,NATIONALITY,SIZE AND TOPTHREEAMERICAN HERE HAVE MISSING VALUES AND ABOVE NOT?????????????
df_new.isnull().sum()

Auction                 0
VehicleAge              0
Make                    0
Model                   0
Trim                    0
Color                   0
Transmission            0
WheelType               0
Nationality             0
Size                    0
TopThreeAmericanName    0
VNZIP1                  0
VNST                    0
VehBCost_bin            0
VehOdo_bin              0
MMR_factor_bin          0
WarrantyCost_bin        0
dtype: int64

In [265]:
#START PATTERN MINING
dt = df_new.groupby('Make')['VehBCost_bin'].apply(list)
dt.head()

Make
ACURA        [[3649.4, 7297.8), [10946.2, 14594.6), [3649.4...
BUICK        [[3649.4, 7297.8), [3649.4, 7297.8), [7297.8, ...
CADILLAC     [[7297.8, 10946.2), [7297.8, 10946.2), [7297.8...
CHEVROLET    [[3649.4, 7297.8), [7297.8, 10946.2), [7297.8,...
CHRYSLER     [[3649.4, 7297.8), [3649.4, 7297.8), [3649.4, ...
Name: VehBCost_bin, dtype: object

In [68]:
baskets = dt.values

In [69]:
baskets[0:4]

array([list([Interval(3649.4, 7297.8, closed='left'), Interval(10946.2, 14594.6, closed='left'), Interval(3649.4, 7297.8, closed='left'), Interval(10946.2, 14594.6, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(3649.4, 7297.8, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(10946.2, 14594.6, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 10946.2, closed='left'), Interval(7297.8, 1094

In [70]:
from fim import apriori

In [71]:
itemsets = apriori(baskets, supp=10, zmin=2, target='a') 

In [72]:
print('Number of itemsets:', len(itemsets))

Number of itemsets: 11


In [73]:
itemsets[:10]

[((Interval(1.0, 3649.4, closed='left'),
   Interval(10946.2, 14594.6, closed='left'),
   Interval(3649.4, 7297.8, closed='left'),
   Interval(7297.8, 10946.2, closed='left')),
  7),
 ((Interval(1.0, 3649.4, closed='left'),
   Interval(10946.2, 14594.6, closed='left'),
   Interval(3649.4, 7297.8, closed='left')),
  7),
 ((Interval(1.0, 3649.4, closed='left'),
   Interval(10946.2, 14594.6, closed='left'),
   Interval(7297.8, 10946.2, closed='left')),
  7),
 ((Interval(1.0, 3649.4, closed='left'),
   Interval(10946.2, 14594.6, closed='left')),
  7),
 ((Interval(1.0, 3649.4, closed='left'),
   Interval(3649.4, 7297.8, closed='left'),
   Interval(7297.8, 10946.2, closed='left')),
  17),
 ((Interval(1.0, 3649.4, closed='left'),
   Interval(3649.4, 7297.8, closed='left')),
  17),
 ((Interval(1.0, 3649.4, closed='left'),
   Interval(7297.8, 10946.2, closed='left')),
  17),
 ((Interval(10946.2, 14594.6, closed='left'),
   Interval(3649.4, 7297.8, closed='left'),
   Interval(7297.8, 10946.2, cl

In [74]:
rules = apriori(baskets, supp=10, zmin=2, target='r', conf=80, 
                report='ascl') 

In [75]:
print('Number of rule:', len(rules))

Number of rule: 14


In [267]:
#EXEMPLE RULES (RULES WITH INDEX 0 - FIRST RULES)
rules[0]

(Interval(7297.8, 10946.2, closed='left'),
 (Interval(1.0, 3649.4, closed='left'),
  Interval(10946.2, 14594.6, closed='left'),
  Interval(3649.4, 7297.8, closed='left')),
 7,
 0.21212121212121213,
 1.0,
 1.1)

In [76]:
count = 0
for r in rules:
    if r[5] > 2 and r[4] > 0.7:
        print(r)
        count += 1
        if count == 10:
            break

In [78]:
#code that filters all ARs with a lift greater than 1.1
#set the library so that it prints the lift
#count to number
count = 0
for r in rules:
    if r[3]>1.1:
        print('%s -- %s' % (str(count), r))
        count = count + 1

In [268]:
print(apriori.__doc__)

apriori (tracts, target='s', supp=10, zmin=1, zmax=None, report='a',
         eval='x', agg='x', thresh=10, prune=None, algo='b', mode='',
         border=None)
Find frequent item sets with the Apriori algorithm.
tracts  transaction database to mine (mandatory)
        The database must be an iterable of transactions;
        each transaction must be an iterable of items;
        each item must be a hashable object.
        If the database is a dictionary, the transactions are
        the keys, the values their (integer) multiplicities.
target  type of frequent item sets to find     (default: s)
        s/a   sets/all   all     frequent item sets
        c     closed     closed  frequent item sets
        m     maximal    maximal frequent item sets
        g     gens       generators
        r     rules      association rules
supp    minimum support of an item set         (default: 10)
        (positive: percentage, negative: absolute number)
conf    minimum confidence of an assoc. rul

In [273]:
rules = apriori(baskets, supp=15, zmin=2, target='r', conf=70, report='YXlC')

In [274]:
rules[0]

(Interval(7297.8, 10946.2, closed='left'),
 (Interval(1.0, 3649.4, closed='left'),
  Interval(10946.2, 14594.6, closed='left'),
  Interval(3649.4, 7297.8, closed='left')),
 90.9090909090909,
 21.21212121212121,
 1.1,
 100.0)

In [279]:
rules = apriori(baskets, supp=15, zmin=2, target='r', conf=70, report='YXLC')

In [280]:
rules[0]

(Interval(7297.8, 10946.2, closed='left'),
 (Interval(1.0, 3649.4, closed='left'),
  Interval(10946.2, 14594.6, closed='left'),
  Interval(3649.4, 7297.8, closed='left')),
 90.9090909090909,
 21.21212121212121,
 110.00000000000001,
 100.0,
 7)